## Homework Module 2: Vector Search

## Q1. What's the minimal value in this array?

In [59]:
from fastembed import TextEmbedding
import numpy as np

query = "I just discovered the course. Can I join now?"
model_name = "jinaai/jina-embeddings-v2-small-en"

# Dense Text Embeddings
embedding_model = TextEmbedding(model_name=model_name)
query_embedded = np.array(list(embedding_model.embed(query)))

print("Mininum value of the vector: ", query_embedded.min())

Mininum value of the vector:  -0.11726373551188797


## Q2. What's the cosine similarity between the vector for the query and the vector for the document?

In [63]:

doc = 'Can I still join the course after the start date?'

# Number of column of the first matrix must be equal to the number of rows of the second matrix
# query_embedded matrix shape -> (1, 512)
# doc_embedded matrix shape -> (1, 512)
# For the current shape, both matrixes are violation the role of matrix multiplication
# To fix it, we can transform one of the matrix into 1D
# doc_embedded (1, 512) -> (512, )
doc_embedded = np.array(list(embedding_model.embed(doc)))[0]

print(query_embedded.shape)
print(doc_embedded.shape)
print("Cosine Similarity: ", query_embedded.dot(doc_embedded)[0])

(1, 512)
(512,)
Cosine Similarity:  0.9008528856818037


## Q3. What's the document index with the highest similarity?


In [ ]:
documents = [{'text': "Yes, even if you don't register, you're still eligible to submit the homeworks.\nBe aware, however, that there will be deadlines for turning in the final projects. So don't leave everything for the last minute.",
  'section': 'General course-related questions',
  'question': 'Course - Can I still join the course after the start date?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Yes, we will keep all the materials after the course finishes, so you can follow the course at your own pace after it finishes.\nYou can also continue looking at the homeworks and continue preparing for the next cohort. I guess you can also start working on your final capstone project.',
  'section': 'General course-related questions',
  'question': 'Course - Can I follow the course after it finishes?',
  'course': 'data-engineering-zoomcamp'},
 {'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
  'section': 'General course-related questions',
  'question': 'Course - When will the course start?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'You can start by installing and setting up all the dependencies and requirements:\nGoogle cloud account\nGoogle Cloud SDK\nPython 3 (installed with Anaconda)\nTerraform\nGit\nLook over the prerequisites and syllabus to see if you are comfortable with these subjects.',
  'section': 'General course-related questions',
  'question': 'Course - What can I do before the course starts?',
  'course': 'data-engineering-zoomcamp'},
 {'text': 'Star the repo! Share it with friends if you find it useful ❣️\nCreate a PR if you see you can improve the text or the structure of the repository.',
  'section': 'General course-related questions',
  'question': 'How can we contribute to the course?',
  'course': 'data-engineering-zoomcamp'}]

cosine_distance = []

for document in documents:
    embedded_text = np.array(list(embedding_model.embed(document["text"])))[0]
    cosine_dist = query_embedded.dot(embedded_text)[0]
    cosine_distance.append(cosine_dist)

max_index = np.argmax(cosine_distance)

print("Document index with the highest similarity: ", max_index)

Document index with the highest similarity:  1


## Q4. Embed question and text fields and compute the cosine distance. What's the highest scoring document?

In [76]:
full_text_cosine_distance = []

for document in documents:
    full_text = document["question"] + " " + document["text"]
    embedded_full_text = np.array(list(embedding_model.embed(full_text)))[0]
    cosine_dist = query_embedded.dot(embedded_full_text)[0]
    full_text_cosine_distance.append(cosine_dist)

max_index = np.argmax(full_text_cosine_distance)

print("Full document text index with the highest similarity: ", max_index)

Full document text index with the highest similarity:  0


## Q5. What's the smallest dimensionality for models in fastembed?

In [82]:
dimensionality = []

for model in TextEmbedding.list_supported_models():
    dimensionality.append(model["dim"])

min_dim = min(dimensionality)

print("Smallest dimensionality: ", min_dim)

Smallest dimensionality:  384


## Q6. After inserting data into Qdrant, what's the highest score in the results?

In [ ]:
import requests

docs_url = 'https://github.com/alexeygrigorev/llm-rag-workshop/raw/main/notebooks/documents.json'
response = requests.get(docs_url)
documents_raw = response.json()

documents = []

for course in documents_raw:
    if course["course"] == "machine-learning-zoomcamp":
        for doc in course["documents"]:
            doc["course"] = course["course"]
            documents.append(doc)


In [ ]:
from qdrant_client import QdrantClient, models
import json

qdrant = QdrantClient("http://localhost:6333")

EMBEDDING_DIMENSIONALITY = 384

for model in TextEmbedding.list_supported_models():
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(json.dumps(model, indent=2))

embedding_model = "BAAI/bge-small-en-v1.5"

In [88]:
collection_name = "machine-learning-zoomcamp"

qdrant.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,
        distance=models.Distance.COSINE
    )
)

True

In [91]:
import uuid

points = []

for document in documents:
    full_text = document["question"] + " " + document["text"]
    point = models.PointStruct(
        id = uuid.uuid4().hex,
        vector=models.Document(text=full_text, model=embedding_model),
        payload={
            "text": document["text"],
            "section": document["section"],
            "course": document["course"]
        }
    )
    points.append(point)

qdrant.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=0, status=<UpdateStatus.COMPLETED: 'completed'>)

In [98]:
results = qdrant.query_points(
    collection_name=collection_name,
    query=models.Document(
        text=query,
        model=embedding_model
    ),
    limit=5,
    with_payload=True
)

score = results.points[0].score

print("Highest score: ", score)

Highest score:  0.73974764
